In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import zipfile
from google.colab import drive

zip_ref = zipfile.ZipFile("/content/drive/My Drive/ct_covid.zip", 'r')
zip_ref.extractall("/content/")
zip_ref.close()

FileNotFoundError: ignored

In [ ]:
import os
import tensorflow as tf
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input,Dense,Reshape,Flatten,Conv2D,Conv2DTranspose,LeakyReLU
from keras.layers import BatchNormalization,Dropout,Embedding,Activation,Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical  

ImportError: ignored

In [ ]:
#@title Default title text
from tensorflow.keras import layers
from tensorflow.keras import Model
#!wget --no-check-certificate \
    #https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    #-O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

#local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = 'imagenet')
#pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = True
  
#pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed4')
#print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
pre_trained_model.layers

In [ ]:
#@title Default title text
from tensorflow.keras.optimizers import RMSprop
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense  (1, activation='sigmoid')(x)           
model = Model( pre_trained_model.input, x) 
model.compile(optimizer = RMSprop(lr=0.00003), #0.0001 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy','Precision', 'Recall'])
model.summary()

In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "/content/dataset/train"
training_datagen = ImageDataGenerator(rescale = 1./255)
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

VALIDATION_DIR = "/content/dataset/test"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
  class_mode='binary',
  batch_size=64
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='binary',
  batch_size=64
)


Found 558 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>0.84 and logs.get('accuracy')>0.90):
      print("\nReached predicted accuracy so cancelling training!")
      self.model.stop_training = True
callbacks = myCallback()

In [ ]:
history = model.fit(
      train_generator,
      validation_data = validation_generator,
      epochs=100,
      verbose=1,
      callbacks = [callbacks]
      )

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, loss, 'r', label='Training loss')
plt.title('Training loss')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Validation loss')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)